<h1>
<center>
Module 5: First look at Bias-Variance Tradeoff
</center>
</h1>
<div class=w19>
<p>
There is a classic problem with machine learning models using labeled data (i.e., supervised learning). Typically you are given training data and you build your model from that. Once you have your model, you can release into the wild where it will start working with new data coming from the world. The problem is that your model can suffer from one of two problems: (1) your model is too weak and underfits the training data or (2) your model is too strong and overfits the training data. The former is called *bias*. High bias can cause a model to miss the relevant relations between columns/features and the target output. Think of our prediction tree that was a stump using only `sex_female` as a column. It ignored all other columns. Think of this predictor as highly biased to liking one column. If it was less biased (more egalitarian) it would have included most if not all the columns.
<p>
  But we saw the strange case of adding more columns causing our scores to go down. What's up with that? It is likely a case of overfitting.
The problem is that the model pays too much attention to the nuances of the training data. It can end up modeling the random noise in the training data. This is called high *variance*. Take the Titanic data. What if a model used the `Name` column as a column to use for splitting? What if we one-hot encoded it? Since every name is unique, we would get 891 new columns. If I built a tree using those 891 columns as splitters and a max-depth of 891, I would get 100% accuracy, right? Convince yourself of that.
<p>
Caveat 1: I am claiming that the *raw* `Name` column has extreme variance. That said, I think it could be useful to wrangle the `Name` column a bit to pull out useful info from the raw values. For instance, I see salutations like Master, Reverend, Miss, Honorable, etc. These indeed might carry information. Maybe they identify passengers of "high class" that were let on the lifeboats. We could wrangle a new binary column `upper_class` that is formed by looking for salutations in the Name column.
<p>
Caveat 2: the drawback of using the Titanic data is that it is hard to see beyond the passengers given to us. Our Titanic models will not be released for future use. There will not be another Titanic built. So maybe better to think of the Loan Table. The models you build for it could definitely be released for future use. If good enough, I suppose they could replace human loan-agents in a bank. You can certainly see where bias might creep in here, e.g., only using the `Married` column to make decisions.
<p>
We will study a variety of methods for handling the Bias-Variance tradeoff in the coming weeks. We are looking for a sweet spot where there is not too much underfitting and not too much overfitting. This week we will try a technique called k-fold cross-validation.
<p>
  Go ahead and load things in.
</div>

In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
with open('/content/gdrive/My Drive/class_tables/titanic_wrangled_week2.csv', 'r') as f:
  titanic_table = pd.read_csv(f)

#Don't need results table. We will build a new one.

titanic_table.head(2)  #make sure it looks ok - we see the results of our week 2 wrangling

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,age_bin,age_Child,age_Adult,age_Senior,sex_female,sex_male,ok_child,pclass_1,pclass_2,pclass_3
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,...,Child,1,0,0,0,1,0,0,0,1
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,...,Adult,0,1,0,1,0,0,1,0,0


In [0]:
pd.set_option('display.max_columns', None)

In [4]:
!rm library_w19_week4b.py

rm: cannot remove 'library_w19_week4b.py': No such file or directory


In [5]:
from google.colab import files
files.upload()

Saving library_w19_week4b.py to library_w19_week4b.py


{'library_w19_week4b.py': b'def predictor_case(row, pred, target):\n\tcase_dict = {(0,0): \'true_negative\', (1,1): \'true_positive\', (0,1): \'false_negative\', (1,0): \'false_positive\'}\n\tactual = row[target]\n\tprediction = row[pred]\n\tcase = case_dict[(prediction, actual)]\n\treturn case\n\ndef accuracy(cases):\n    tp = cases[\'true_positive\']\n    tn = cases[\'true_negative\']\n    fp = cases[\'false_positive\']\n    fn = cases[\'false_negative\']\n    return (tp + tn)/(tp+tn+fp+fn)\n\ndef f1(cases):\n    #the heart of the matrix\n    tp = cases[\'true_positive\']\n    fn = cases[\'false_negative\']\n    tn = cases[\'true_negative\']\n    fp = cases[\'false_positive\']\n    \n    #other measures we can derive\n    recall = 1.0*tp/(tp+fn)  # positive correct divided by total positive in the table\n    precision = 1.0*tp/(tp+fp) # positive correct divided by all positive predictions made\n    \n    #now for the one we want\n    f1 = 2/(1/recall + 1/precision)\n    \n    return 

In [6]:
from library_w19_week4b import *

%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 f1	 find_best_splitter	 generate_table	 gig	 gini	 
informedness	 path_id	 predictor_case	 probabilities	 reorder_paths	 tree_predictor	 


<h1>Extreme example of overfitting</h1>

Let's look at my idea of using the `Name` column from the titanic table as a splitter. First I will one hot encode the `Name` column.

In [0]:
one_hot_name = pd.get_dummies(titanic_table['Name'],prefix='z',dummy_na=False)  # false because should not have empties
big_table = titanic_table.join(one_hot_name)

In [8]:
big_table.head()  #Yikes - look at all those new columns!

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,no_age,filled_age,emb_C,emb_Q,emb_S,emb_nan,age_bin,age_Child,age_Adult,age_Senior,sex_female,sex_male,ok_child,pclass_1,pclass_2,pclass_3,"z_Abbing, Mr. Anthony","z_Abbott, Mr. Rossmore Edward","z_Abbott, Mrs. Stanton (Rosa Hunt)","z_Abelson, Mr. Samuel","z_Abelson, Mrs. Samuel (Hannah Wizosky)","z_Adahl, Mr. Mauritz Nils Martin","z_Adams, Mr. John","z_Ahlin, Mrs. Johan (Johanna Persdotter Larsson)","z_Aks, Mrs. Sam (Leah Rosen)","z_Albimona, Mr. Nassef Cassem","z_Alexander, Mr. William","z_Alhomaki, Mr. Ilmari Rudolf","z_Ali, Mr. Ahmed","z_Ali, Mr. William","z_Allen, Miss. Elisabeth Walton","z_Allen, Mr. William Henry","z_Allison, Master. Hudson Trevor","z_Allison, Miss. Helen Loraine","z_Allison, Mrs. Hudson J C (Bessie Waldo Daniels)","z_Allum, Mr. Owen George","z_Andersen-Jensen, Miss. Carla Christine Nielsine","z_Anderson, Mr. Harry","z_Andersson, Master. Sigvard Harald Elias","z_Andersson, Miss. Ebba Iris Alfrida","z_Andersson, Miss. Ellis Anna Maria","z_Andersson, Miss. Erna Alexandra","z_Andersson, Miss. Ingeborg Constanzia","z_Andersson, Miss. Sigrid Elisabeth","z_Andersson, Mr. Anders Johan","z_Andersson, Mr. August Edvard (""Wennerstrom"")","z_Andersson, Mrs. Anders Johan (Alfrida Konstantia Brogren)","z_Andreasson, Mr. Paul Edvin","z_Andrew, Mr. Edgardo Samuel","z_Andrews, Miss. Kornelia Theodosia","z_Andrews, Mr. Thomas Jr","z_Angle, Mrs. William A (Florence ""Mary"" Agnes Hughes)","z_Appleton, Mrs. Edward Dale (Charlotte Lamson)","z_Arnold-Franchi, Mr. Josef","z_Arnold-Franchi, Mrs. Josef (Josefine Franchi)","z_Artagaveytia, Mr. Ramon","z_Asim, Mr. Adola","z_Asplund, Master. Clarence Gustaf Hugo","z_Asplund, Master. Edvin Rojj Felix","z_Asplund, Miss. Lillian Gertrud","z_Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)","z_Astor, Mrs. John Jacob (Madeleine Talmadge Force)","z_Attalah, Miss. Malake","z_Attalah, Mr. Sleiman","z_Aubart, Mme. Leontine Pauline","z_Augustsson, Mr. Albert","z_Ayoub, Miss. Banoura","z_Backstrom, Mr. Karl Alfred","z_Backstrom, Mrs. Karl Alfred (Maria Mathilda Gustafsson)","z_Baclini, Miss. Eugenie","z_Baclini, Miss. Helene Barbara","z_Baclini, Miss. Marie Catherine","z_Baclini, Mrs. Solomon (Latifa Qurban)","z_Badt, Mr. Mohamed","z_Bailey, Mr. Percy Andrew","z_Balkic, Mr. Cerin","z_Ball, Mrs. (Ada E Hall)","z_Banfield, Mr. Frederick James","z_Barah, Mr. Hanna Assi","z_Barbara, Miss. Saiide","z_Barbara, Mrs. (Catherine David)","z_Barber, Miss. Ellen ""Nellie""","z_Barkworth, Mr. Algernon Henry Wilson","z_Barton, Mr. David John","z_Bateman, Rev. Robert James","z_Baumann, Mr. John D","z_Baxter, Mr. Quigg Edmond","z_Baxter, Mrs. James (Helene DeLaudeniere Chaput)","z_Bazzani, Miss. Albina","z_Beane, Mr. Edward","z_Beane, Mrs. Edward (Ethel Clarke)","z_Beavan, Mr. William Thomas","z_Becker, Master. Richard F","z_Becker, Miss. Marion Louise","z_Beckwith, Mr. Richard Leonard","z_Beckwith, Mrs. Richard Leonard (Sallie Monypeny)","z_Beesley, Mr. Lawrence","z_Behr, Mr. Karl Howell","z_Bengtsson, Mr. John Viktor","z_Berglund, Mr. Karl Ivar Sven","z_Berriman, Mr. William John","z_Betros, Mr. Tannous","z_Bidois, Miss. Rosalie","z_Bing, Mr. Lee","z_Birkeland, Mr. Hans Martin Monsen","z_Bishop, Mr. Dickinson H","z_Bishop, Mrs. Dickinson H (Helen Walton)","z_Bissette, Miss. Amelia","z_Bjornstrom-Steffansson, Mr. Mauritz Hakan","z_Blackwell, Mr. Stephen Weart","z_Blank, Mr. Henry","z_Bonnell, Miss. Elizabeth","z_Bostandyeff, Mr. Guentcho","z_Boulos, Miss. Nourelain","z_Boulos, Mr. Hanna","z_Boulos, Mrs. Joseph (Sultana)","z_Bourke, Miss. Mary","z_Bourke, Mr. John","z_Bourke, Mrs. John (Catherine)","z_Bowen, Mr. David John ""Dai""","z_Bowerman, Miss. Elsie Edith","z_Bracken, Mr. James H","z_Bradley, Mr. George (""George Arthur Brayton"")","z_Braund, Mr. Lewis Richard","z_Braund, Mr. Owen Harris","z_Brewe, Dr. Arthur Jackson","z_Brocklebank, Mr. William Alfred","z_Brown, Miss. Amelia ""Mildred""","z_Brown, Mr. Thomas William Solo

<h2>We just added 891 columns!</h2>

In essence, we have a column for every row. If we build our tree correctly, we should get 100% accuracy. Let's build the tree. I only have to use the 891 columns as splitters. I added a `z` prefix to make them easy to find.

In [9]:
splitter_columns = []
for col in big_table.columns.values:
  if 'z_' in col:
    splitter_columns.append(col)
    
len(splitter_columns)

891

In [10]:
splitter_columns[:5]  #looks like pandas alphabetized them for me

['z_Abbing, Mr. Anthony',
 'z_Abbott, Mr. Rossmore Edward',
 'z_Abbott, Mrs. Stanton (Rosa Hunt)',
 'z_Abelson, Mr. Samuel',
 'z_Abelson, Mrs. Samuel (Hannah Wizosky)']

I am going to bail out a bit here. Normally I would build a tree with the 891 splitter columns and a depth of 891. But that takes awhile to run. Instead I'll build a tree with the first 5 columns and then take a look at the paths thar are produced.

In [0]:
tree_name_5 = build_tree_iter(big_table, splitter_columns[:5], 'Survived', {'max-depth': 5})


In [12]:
tree_name_5['paths'][0]  #Look at path 0.

{'conjunction': [('z_Abbott, Mrs. Stanton (Rosa Hunt)_1',
   <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
 'gig_score': 0.0008531576117549178,
 'prediction': 1}

You can see that this path has one predicate. It asks if the `Name` value is equal to 'Abbott, Mrs. Stanton (Rosa Hunt)'. If it is true then prediction is 1. Is this prediction always correct? Yes! When I was building the tree, I just looked at the `Survived` column for Mrs. Abbott and used that. And there is only 1 row that matches the predicate.
<p>
  Let's look at the next path.

In [13]:
tree_name_5['paths'][1]

{'conjunction': [('z_Abbott, Mrs. Stanton (Rosa Hunt)_0',
   <function library_w19_week4b.build_pred.<locals>.<lambda>>),
  ('z_Abelson, Mrs. Samuel (Hannah Wizosky)_1',
   <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
 'gig_score': 0.000856037737108517,
 'prediction': 1}

This path has 2 predicates: (1) Name is not 'Abbott, Mrs. Stanton (Rosa Hunt)', and (2) Name is 'Abelson, Mrs. Samuel (Hannah Wizosky)'. If both are true pedict 1. Again, this is always a perfect prediction.
<p>
  I'll do one more.

In [14]:
tree_name_5['paths'][2]

{'conjunction': [('z_Abbott, Mrs. Stanton (Rosa Hunt)_0',
   <function library_w19_week4b.build_pred.<locals>.<lambda>>),
  ('z_Abelson, Mrs. Samuel (Hannah Wizosky)_0',
   <function library_w19_week4b.build_pred.<locals>.<lambda>>),
  ('z_Abbing, Mr. Anthony_1',
   <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
 'gig_score': 0.0003294362168793641,
 'prediction': 0}

Has 3 predicates. Rule out the first 2 names and check the 3rd name. I hope you can see that we are building the tree below. Given a row we are trying to predict, we will eventually come to the `z_`  column that matches and always choose the right prediction.

<img src='https://www.dropbox.com/s/eycivebhtbu06vx/Screenshot%202019-01-28%2015.31.54.png?raw=1'>

So what's wrong with that? We have 100% accuracy. The problem is that when you see new passengers, their names will not match any of the nodes in the tree. What will happen then? We will end up on the False branch of the last node in the tree. And we know from the way we do tree building, we will have a problem. I'll let you figure it out for extra credit.

<h2>How can I test overfitting?</h2>

The key issue in overfitting your tree is using the same data to both build the tree and test the tree. One way to check overfitting is to modify how we build and test. How about dividing up the full table into a bulding sub-table (typically called a training table) and a testing sub-table? We will try that next.

<h1>First let's build a new results table</h1>

Before trying the split idea, let's do it our old-school way: build and test from full table. It will give us a benchmark to compare against.
<p>

In [0]:
hypers_1 = {'max-depth': 3, 'gig-cutoff': 0.0}

Here are splitter columns I will use.

In [0]:
splitter_columns = [
 'emb_C',
 'emb_Q',
 'emb_S',
 'emb_nan',
 'age_Child',
 'age_Adult',
 'age_Senior',
 'no_age',
 'ok_child',
 'sex_female',
 'sex_male', 
 'pclass_1',
 'pclass_2',
 'pclass_3'
]

In [17]:
tree3 = build_tree_iter(titanic_table, splitter_columns, 'Survived', hypers_1)
print(len(tree3['paths']))

8


What I want to keep track of are the scores for the 3 measures: accuracy, f1, informedness. I'll build a new table for this. I will also include a name column to give some description of what we are doing.

In [18]:
cols = ['name', 'accuracy', 'f1', 'informedness']
kfold_table_1 = pd.DataFrame(columns=cols)  # empty for now
kfold_table_1

,name,accuracy,f1,informedness


I am going to do one more thing. I am going to add a type of comment to the kfold_table_1 dataframe. This will help me remember what settings I was using to get the table results.
<p>
  How did I come up with this code? Stackoverflow as usual: https://stackoverflow.com/a/54137536/4996152. Note that this is still a bit of a kludge. When I write the table out to a csv file, I'll lose this meta information. And we will see it is even worse than that in a minute.

In [19]:
from types import SimpleNamespace

kfold_table_1.meta = SimpleNamespace()
kfold_table_1.meta.hypers = hypers_1
kfold_table_1.meta

namespace(hypers={'max-depth': 3, 'gig-cutoff': 0.0})

Here is where we would start computing predictions, types and scores. I only care about scores so I think we can automate a big portion of what we need.

In [0]:
def produce_scores(table, tree, target):
    scratch_table = pd.DataFrame(columns=['prediction', 'actual'])
    scratch_table['prediction'] = table.apply(lambda row: tree_predictor(row, tree), axis=1)
    scratch_table['actual'] = table[target]  # just copy the target column
    cases = scratch_table.apply(lambda row: predictor_case(row, pred='prediction', target='actual'), axis=1)
    vc = cases.value_counts()
    return [accuracy(vc), f1(vc), informedness(vc)]


Let's try it out on the full table and tree3.


In [21]:
scores_3 = produce_scores(titanic_table, tree3, 'Survived')
scores_3


[0.8237934904601572, 0.7288428324697754, 0.5696002300834051]

Append to our results table.

In [22]:
kfold_table_1 = kfold_table_1.append({'name':'full_table',
                                      'accuracy': scores_3[0],
                                      'f1': scores_3[1],
                                      'informedness': scores_3[2]}, ignore_index=True)
kfold_table_1

,name,accuracy,f1,informedness
0,full_table,0.823793,0.728843,0.5696


In [0]:
#You will get an error if you execute this code. Why? Because pandas does not do what we want.
#When pandas builds a new table it does not copy over the meta data. It is an issue that has
#been rasied with pandas' developers.

kfold_table_1.meta  #should produce error of missing meta attribute

<h2>
Using both a training set and a testing set
</h2>
<div class=h1_cell>
<p>
Now let's try a new approach. We will split the full table into a training/building set and a testing set.

The standard approach is to take 2/3 of the data as training and hold out 1/3 as testing.
</div>

In [24]:
total_len = len(titanic_table)
split_boundary = int(total_len*(2/3))
split_boundary

594


<div class=h1_cell>
<p>
You can use a slice operator on a table just like you can on a list. Cool.
</div>

In [25]:
training_table = titanic_table[0:split_boundary]  # 0-593
test_table = titanic_table[split_boundary:]       # 594 to 890
print(len(training_table))
print(len(test_table))

594
297


<div class=h1_cell>
Now let's again build a tree like we did above with same splitter columns and same hypers. But now we will use the training table to build the tree.
</div>

In [26]:
#Notice using training_table not titanic_table

tree_train = build_tree_iter(training_table, splitter_columns, 'Survived', hypers_1)
print(len(tree_train['paths']))
#tree_train['paths']

8


Now to testing. Let's check out scores.

In [27]:
#Notice test_table and not titanic_table

produce_scores(test_table, tree_train, 'Survived')

[0.8047138047138047, 0.6627906976744187, 0.4907407407407407]

In [28]:
kfold_table_1  #Check against results for full table which we have stored away


,name,accuracy,f1,informedness
0,full_table,0.823793,0.728843,0.5696


We dropped on all scores. This is very typical. When testing with data different than training data, scores drop. Why? We do less overfitting of training data and hence get a more realistic score.

<h2>
Ok, let's generalize
</h2>
<p>
<div class=h1_cell>
<p>
What we are doing is called cross-validation: breaking our data up into training and testing sets. I'd like to push a bit harder on the cross-validation idea. I suggest we try more than just one split for training/testing. Try a bunch then average their results. There are many ways we can consider generating a separate set of splits. I am going to focus on a standard approach called K-Folding. The general idea is that we divide the table into K partitions or folds. We then build K trees from various combinations of the folds and do K tests, one for each tree. Where does K come from? You get to choose. I will use K=5 below but K=10 is more common.
<p>
Even with this standard algorithm there are variations on how you select the K folds. I am going to use a sequential approach, splitting into folds along the row indices. So my first fold will be from 0 to i, my next fold from i+1 to j, etc.
<p>
I'll also refer to the folds as slices.
</div>

In [29]:
k = 5  # more often 10

total_len = len(titanic_table.index)
slice_size = int(1.0/k*total_len)
slice_size

178

In [0]:
slice_1 = titanic_table[0:slice_size]
slice_2 = titanic_table[1*slice_size:2*slice_size]
slice_3 = titanic_table[2*slice_size:3*slice_size]
slice_4 = titanic_table[3*slice_size:4*slice_size]
slice_5 = titanic_table[4*slice_size:]

<div class=h1_cell>
Now that I have my 5 folds/slices, I'll take the first step: train on 4 of the slices and test on the remaining slice. 
</div>

In [31]:
fold1_test_table = slice_1
fold1_train_table = pd.concat([slice_2, slice_3, slice_4, slice_5])
len(fold1_train_table)

713

<div class=h1_cell>
Keep plodding along until I have 5 training sets and 5 test sets. 
</div>

In [32]:
fold2_test_table = slice_2
fold2_train_table = pd.concat([slice_1, slice_3, slice_4, slice_5])
len(fold2_train_table)

713

In [33]:
fold3_test_table = slice_3
fold3_train_table = pd.concat([slice_1, slice_2, slice_4, slice_5])
len(fold3_train_table)

713

In [34]:
fold4_test_table = slice_4
fold4_train_table = pd.concat([slice_1, slice_2, slice_3, slice_5])
len(fold4_train_table)

713

In [35]:
fold5_test_table = slice_5
fold5_train_table = pd.concat([slice_1, slice_2, slice_3, slice_4])
len(fold5_train_table)

712

<h2>
Try fold-set number 1
</h2>
<p>
<div class=h1_cell>
Whew. We now have 5 pairs of training and test. I'll try the first fold and see how it goes. First I'll build a tree from the training slice then test it with the test slice.

</div>

In [0]:
fold1_tree = build_tree_iter(fold1_train_table, splitter_columns, 'Survived', hypers_1)  # train

In [37]:
fold1_scores = produce_scores(fold1_test_table, fold1_tree, 'Survived')  # test
fold1_scores

[0.8089887640449438, 0.6666666666666666, 0.5006409343398377]

<h2>
Do the remaining 4
</h2>
<p>
<div class=h1_cell>


</div>

In [38]:
fold2_tree = build_tree_iter(fold2_train_table, splitter_columns, 'Survived', hypers_1)  # train
fold2_scores = produce_scores(fold2_test_table, fold2_tree, 'Survived')  # test
fold2_scores

[0.8033707865168539, 0.7328244274809159, 0.5653846153846154]

In [39]:
fold3_tree = build_tree_iter(fold3_train_table, splitter_columns, 'Survived', hypers_1)  # train
fold3_scores = produce_scores(fold3_test_table, fold3_tree, 'Survived')  # test
fold3_scores

[0.8370786516853933, 0.7603305785123966, 0.6108465608465607]

In [40]:
fold4_tree = build_tree_iter(fold4_train_table, splitter_columns, 'Survived', hypers_1)  # train
fold4_scores = produce_scores(fold4_test_table, fold4_tree, 'Survived')  # test
fold4_scores

[0.7808988764044944, 0.6608695652173913, 0.47913650125049356]

In [41]:
fold5_tree = build_tree_iter(fold5_train_table, splitter_columns, 'Survived', hypers_1)  # train
fold5_scores = produce_scores(fold5_test_table, fold5_tree, 'Survived')  # test
fold5_scores

[0.8268156424581006, 0.7102803738317757, 0.5502717391304348]

<h2>Take the average</h2>

Finally, let's take the average of the 5 folds. You can see I am using `np.add` and `reduce` in my averaging code. It is probably the case that a list-comprehension using `zip` would be clearer. I like both `numpy` and `reduce` because of potential for parallelism if I think I might need a performance boost in the future.

In [45]:
import numpy as np
from functools import reduce

average_1 = tuple(reduce(np.add, (fold2_scores, fold3_scores, fold4_scores, fold5_scores), fold1_scores)/5)  #take average of 5 folds
average_1

(0.8114305442219572, 0.7061943223418292, 0.5412560701903883)

<h2>
Kind of tedious
</h2>
<p>
<div class=h1_cell>
It was kind of ok building the 5 separate folds by hand, but when I move to K=10, it's a bit much to repeat all this 10 times for the 10 folds. So let's build a new function to do it for us. First, I'll define a helper function that given a list of slices and an index, will create a new list of slices with the index slice left out. Why? Because we need something to compute the training table and will use this function to get the necessary slices.
</div>

In [0]:
def compute_training(slices, left_out):
    training_slices = []
    for i,slice in enumerate(slices):
        if i == left_out:
            continue
        training_slices.append(slices[i])
    return pd.concat(training_slices)  # note we are returning a table (DataFrame)


In [49]:
#test it

a_traing_table = compute_training((slice_1,slice_2,slice_3,slice_4,slice_5), 4)  #leave out 4th (i.e., last) slice
len(a_traing_table)  #expect 891 minus the last slice = 712


712

<h2>
Ready to automate
</h2>
<p>
<div class=h1_cell>
I'll define a function that takes as arguments (a) the big table, (b) value for K, (c) the target column, (e) the hyper parameters to be used in building a model, and (f) the candidate columns to build the splitters from.
<p>
The function's output will be a results table. Notice I also added a comment onto the table to help me remember what hyper parameteres I was using.
</div>

In [0]:
def k_fold(table, k, target, hypers, candidate_columns):
  
    #set up the table where we will record fold results
    result_columns = ['name',  'accuracy', 'f1', 'informedness']
    k_fold_results_table = pd.DataFrame(columns=result_columns)
    
    #generate the slices
    total_len = len(table.index)
    slice_size = int(total_len/(1.0*k))
    slices = []
    for i in range(k-1):
        a_slice =  table[i*slice_size:(i+1)*slice_size]
        slices.append( a_slice )
    slices.append( table[(k-1)*slice_size:] )  # whatever is left
    
    #generate test results
    all_scores = []  #keep track of all k results
    for i in range(k):
        test_table = slices[i]
        train_table = compute_training(slices, i)
        fold_tree = build_tree_iter(train_table, candidate_columns, target, hypers)  # train
        scores = produce_scores(test_table, fold_tree, target)  # test
        results_row = {'name': 'fold_'+str(i), 'accuracy': scores[0], 'f1': scores[1], 'informedness': scores[2]}
        k_fold_results_table = k_fold_results_table.append(results_row,ignore_index=True)
        all_scores.append(scores)
    
    #compute average of all folds
    avg_scores = tuple(reduce(lambda total, triple: np.add(triple, total), all_scores)/5)
    results_row = {'name': 'average', 'accuracy': avg_scores[0], 'f1': avg_scores[1], 'informedness': avg_scores[2]}
    k_fold_results_table = k_fold_results_table.append(results_row,ignore_index=True)
    
    #note that I add the meta comment as last step to avoid it being wiped out
    k_fold_results_table.meta = SimpleNamespace()
    k_fold_results_table.meta.hypers  = hypers # adds comment to remind me of hyper params used
    
    return k_fold_results_table


<h2>
Let's try it out with default hyper params
</h2>
<p>
<div class=h1_cell>
I'll use K=5 and default values for hyper-parameters, i.e., max-depth of 4, gig-cutoff of 0.
</div>

In [0]:
default5_table = k_fold(titanic_table, 5, 'Survived', {}, splitter_columns)  # max-depth=4


In [52]:

default5_table


,name,accuracy,f1,informedness
0,fold_0,0.808989,0.666667,0.500641
1,fold_1,0.820225,0.761194,0.603846
2,fold_2,0.837079,0.760331,0.610847
3,fold_3,0.780899,0.660870,0.479137
4,fold_4,0.843575,0.745455,0.597147
5,average,0.818153,0.718903,0.558323


In [53]:
default5_table.meta  #using defaults

namespace(hypers={})

<h2>
Tuning hyper-parameters
</h2>
<p>
<div class=h1_cell>
We have 2 hyper-parameters. I'll concentrate on max-depth. I have the results for the default max-depth (i.e., 4) above. The means from the 5 folds are (0.818153,	0.718903,	0.558323).
<p>
I'll now try changing the max-depth to 3 and see how we do. As reminder, we did this in the last module and got a result that was the same as with max-depth 4. But that was using same data for training and testing. Now let's see if K-folding gives us a different answer.
</div>

In [55]:
max3_table = k_fold(titanic_table, 5, 'Survived', {'max-depth':3}, splitter_columns)
max3_table


,name,accuracy,f1,informedness
0,fold_0,0.808989,0.666667,0.500641
1,fold_1,0.803371,0.732824,0.565385
2,fold_2,0.837079,0.760331,0.610847
3,fold_3,0.780899,0.660870,0.479137
4,fold_4,0.826816,0.710280,0.550272
5,average,0.811431,0.706194,0.541256


<div class=h1_cell>
<p>
Here are our means from depth 4: (0.818153	0.718903	0.558323).
<p>
Here are our means from depth 3: (0.811431	0.706194	0.541256).
<p>
Using 5-folds, we lost a little ground when going from 4 to 3.
<p>
Let's try depth 2. Note that by decreasing the depth, we are moving the needle away from high variance but towards high bias.
</div>

In [58]:
max2_table = k_fold(titanic_table, 5, 'Survived', {'max-depth':2}, splitter_columns)
max2_table

,name,accuracy,f1,informedness
0,fold_0,0.752809,0.511111,0.322604
1,fold_1,0.764045,0.650000,0.470000
2,fold_2,0.792135,0.733813,0.561905
3,fold_3,0.741573,0.646154,0.432671
4,fold_4,0.798883,0.704918,0.541440
5,average,0.769889,0.649199,0.465724


<div class=h1_cell>
Lost quite a bit of ground with level 2.
</div>

<h2>
Where now?
</h2>
<p>
<div class=h1_cell>
<p>
We could try a few more values of depth. But we also could start playing with the other knob, the gig cutoff. By the end of our exploration, we should have a good idea on what values to set our hyper-parameters to. At that point, we will generate the final tree using all of the data. Something like this:
<p>
<pre>
<code>
optimal_depth = ...  # what we discovered in our K-folding
optimal_gig_cutoff = ...  # ditto
hypers = {'max-depth': optimal_depth, 'gig-cutoff': optimal_gig_cutoff )
final_tree = build_tree_iter(titanic_table, candidate_columns, 'Survived', hypers )
</code>
</pre>
</div>

<hr>
<h1>Write it out</h1>
<div class=h1_cell>

We did not change the tree so no need to write it out.
  <p>
    We did add functions so add them to your library and store under name `library_w19_week5.py`. Reminder: if one of your functions in library_w19_week5.py imports a library, that import has to happen in library_w19_week5.py. The functions in your library cannot see the imports you do in the notebook. Separate namespace and all that.
</div>


<h2>
Next up
</h2>
<p>
<div class=h1_cell>
Think about this. With K-folding, we are building k separate trees but then throwing them away. The final tree is produced from all the data. What if we decided not to throw those k trees away? What if we chose to keep all the trees as the "final tree". We would have an ensemble of trees (AKA a forest). How would they agree among themselves on the correct prediction? How about simply letting them vote. That's what is coming up next.
</div>